<a href="https://colab.research.google.com/github/souradip93/CS69002_9A_18CS60R07/blob/master/Assignment9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import the libraries**

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import nltk
import re

# **Upload Train file**

Option 1 - Upload the trainig dataset

In [0]:
from google.colab import files
uploaded = files.upload()
df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.txt'].decode('utf-8')), sep='\t')
df_train.head()


uploaded = files.upload()
df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.txt'].decode('utf-8')), sep='\t')
df_test.head()

*OR*

Option 2 - Store the file in your google drive account

In [3]:
from google.colab import drive
drive.mount('/content/drive')
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Train_20K.txt', sep='\t')
df_train.head()
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Test_5K.txt', sep='\t')
df_test.head()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


,text,label
0,as always this is an inaccurate picture of the...,0
1,Did the movie-makers even preview this before ...,0
2,"Heavily re-edited and often confusing, the ori...",0
3,I notice that most of the people who think thi...,0
4,"First of all, this is a low-budget movie, so m...",0


In [0]:
train_data_X = df_train['text'].astype(str).tolist()
train_data_Y = df_train['label'].astype(int)

In [0]:
test_data_X = df_test['text'].astype(str).tolist()
test_data_Y = df_test['label'].astype(int)

# **Preprocessing**

1. Remove br tags and numbers

In [0]:
def remove_br_tags_and_numbers(train_data_X):
  import re
  regex = re.compile(r'<br.*?>|\d+')
  train_data_X = [regex.sub('', x) for x in train_data_X]
  return train_data_X

2. Tokenize - Consider only alphanumeric characters

In [0]:

def tokenize(train_data_X):
  from nltk.tokenize import RegexpTokenizer
  tokenizer = RegexpTokenizer(r'\w+')
  train_data_X = [tokenizer.tokenize(x) for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X


3. Convert to lower case

In [0]:
def convert_to_lower(train_data_X):
  train_data_X = [[y.lower() for y in x] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

4. Remove stopwords

In [0]:
def remove_stopwords(train_data_X):
  from nltk.corpus import stopwords
  nltk.download('stopwords')
  stopword_set = set(stopwords.words('english'))
  train_data_X = [[y for y in x if y not in stopword_set] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

5. Perform lemmatization

In [0]:
def lemmatize(train_data_X):
  from nltk.stem import WordNetLemmatizer 
  nltk.download('wordnet')
  lemmatizer = WordNetLemmatizer()
  train_data_X = [[lemmatizer.lemmatize(y) for y in x] for x in train_data_X]
  return train_data_X

**Preprocessing on Training data**

In [11]:
train_data_X = remove_br_tags_and_numbers(train_data_X)
train_data_X = tokenize(train_data_X)
train_data_X = convert_to_lower(train_data_X)
train_data_X = remove_stopwords(train_data_X)
train_data_X = lemmatize(train_data_X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


**Preprocessing on Test data**

In [12]:
test_data_X = remove_br_tags_and_numbers(test_data_X)
test_data_X = tokenize(test_data_X)
test_data_X = convert_to_lower(test_data_X)
test_data_X = remove_stopwords(test_data_X)
test_data_X = lemmatize(test_data_X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#**Reduce Vocab**

In [0]:
def generate_word_frequency(dataset):
  word_frequency = {}
  for sent in dataset:
      for word in sent:
          if word not in word_frequency:
              word_frequency[word] = 1
          else:
              val = word_frequency[word]
              val += 1
              word_frequency[word] = val
              
  #word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_frequency

In [0]:
THRESHOLD_FREQUENCY = 10

def reduce_vocab(word_frequency):
  word_to_ix = {}
  for word in word_frequency:
    val = word_frequency[word]
    if val >= THRESHOLD_FREQUENCY:
      word_to_ix[word] = len(word_to_ix)
  word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_to_ix

In [22]:
word_frequency = generate_word_frequency(train_data_X + test_data_X)
word_to_ix = reduce_vocab(word_frequency)

VOCAB_SIZE = len(word_to_ix)
print(VOCAB_SIZE, word_to_ix)

17043 {'john': 0, 'water': 1, 'given': 2, 'u': 3, 'genuinely': 4, 'enjoyable': 5, 'film': 6, 'certainly': 7, 'without': 8, 'shocking': 9, 'esque': 10, 'moment': 11, 'tamer': 12, 'older': 13, 'stuff': 14, 'pink': 15, 'pecker': 16, 'back': 17, 'early': 18, 'mainstream': 19, 'stage': 20, 'reminds': 21, 'viewer': 22, 'kind': 23, 'humor': 24, 'evident': 25, 'polyester': 26, 'overall': 27, 'really': 28, 'fun': 29, 'comedy': 30, 'great': 31, 'first': 32, 'two': 33, 'season': 34, 'series': 35, 'strange': 36, 'funny': 37, 'drama': 38, 'element': 39, 'bill': 40, 'mother': 41, 'struggling': 42, 'usual': 43, 'problem': 44, 'life': 45, 'bit': 46, 'depressing': 47, 'mix': 48, 'well': 49, 'th': 50, 'probably': 51, 'dropped': 52, 'soon': 53, 'became': 54, 'one': 55, 'funniest': 56, 'bbc': 57, 'ever': 58, 'made': 59, 'chemistry': 60, 'ben': 61, 'character': 62, 'always': 63, 'many': 64, 'brilliant': 65, 'memorable': 66, 'sketch': 67, 'christmas': 68, 'special': 69, 'hilarious': 70, 'real': 71, 'treat':

Store the word index once created

In [0]:
import pickle

pickle_out = open("word_to_ix.pickle","wb")
pickle.dump(word_to_ix, pickle_out)
pickle_out.close()

from google.colab import files
model = files.download("word_to_ix.pickle")


Irrelevant for this Task. Remove it later

In [24]:
label_to_ix = {1: 1, 0: 0}
ix_to_label = {v: k for k, v in label_to_ix.items()}

label_to_ix, ix_to_label

({0: 0, 1: 1}, {0: 0, 1: 1})

#**Pytorch initialization**

In [39]:
import torch.nn as nn
import torch.nn.functional as F

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.device('cuda')



device(type='cuda')

#**Create bow vectors**

In [0]:
from torch.autograd import Variable
import time

def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[ word_to_ix['UNKNOWN'] ] += 1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    #print(label)
    return torch.LongTensor([label_to_ix[label]])

bow_vec_orig = []
for instance in train_data_X:
  ins_bow_vector = make_bow_vector(instance, word_to_ix)
  bow_vec_orig.append(Variable(ins_bow_vector).cuda())
  
label_orig = []
for l in train_data_Y:
  label_vector = make_target(l, label_to_ix)
  label_orig.append(Variable(label_vector).cuda())

#**Neural network for Task 1**

In [0]:
class Task1(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons, vocab_size):
    super(Task1, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons)
    self.lin2 = nn.Linear(hidden_layer_neurons, num_labels)
    
  def forward(self, x):
    out = self.lin1(x)
    return F.softmax(self.lin2(out))

Setting model parameters for Task 1 A

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS = 50
  
task1a = Task1(NUM_LABELS, HIDDEN_LAYER_NEURONS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function1a = nn.NLLLoss()
opt1a = torch.optim.SGD(task1a.parameters(), lr = 0.1)

Setting model parameters for Task 1 B

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS = 100
  
task1b = Task1(NUM_LABELS, HIDDEN_LAYER_NEURONS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function1b = nn.NLLLoss()
opt1b = torch.optim.SGD(task1b.parameters(), lr = 0.1)

Setting model parameters for Task 1 C

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS = 150
  
task1c = Task1(NUM_LABELS, HIDDEN_LAYER_NEURONS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function1c = nn.NLLLoss()
opt1c = torch.optim.SGD(task1c.parameters(), lr = 0.1)

#**Training for Task 1**

**Task 1 a**

*TRAINING*

In [55]:
bow_vec = bow_vec_orig[:]
label = label_orig[:]
# the training loop
for epoch in range(10):
    start = time.time()
    for ind, instance in enumerate(train_data_X):
        task1a.zero_grad()
        probs = task1a(bow_vec[ind]) # forward pass
        loss = loss_function1a(probs, label[ind])
        loss.backward()
        opt1a.step()
    end = time.time()
    print('Time Taken - ' + str(end - start))
    print('Epoch - ' + str(epoch) + ' , LOSS - ' + str(loss.data))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Time Taken - 16.533740043640137
Epoch - 0 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.262680768966675
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.28729009628296
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.668848037719727
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.500035762786865
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.223926544189453
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.265756607055664
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.036043643951416
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.28712797164917
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.213910818099976
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')


*TESTING*

In [58]:
print('--- AFTER TRAINING ---')
countTotalCorrect = 0
countActualPositives = 0
countPredictedPostives = 0
countPredictedNegatives = 0

for ind, instance in enumerate(test_data_X):
    bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
    logprobs = task1a(bow_vec)
    pred = np.argmax(logprobs.cpu().data.numpy())
    label = test_data_Y[ind]
    if label == 1:
      countActualPositives += 1
      if ix_to_label[pred] == 1:
        countPredictedPostives += 1
      else:
        countPredictedNegatives += 1
      
    if label == ix_to_label[pred]:
      countTotalCorrect += 1
      
print('Accuracy - ' + str(float(countTotalCorrect)/ len(test_data_X)))
print('Recall - ' + str(float(countPredictedPostives)/ countActualPositives))
#print('Precision - ' + str(float(countPredictedPostives)/ (countPredictedPostives + countPredictedNegatives)))

--- AFTER TRAINING ---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Accuracy - 0.8362
Recall - 0.8542494042891183
Precision - 0.8542494042891183


Save the model

In [61]:
import torch
torch.save(task1a,'model1a_gpu.json')

from google.colab import files
model = files.download("model1a_gpu.json")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


***Task 1 b***

*TRAINING*

In [64]:
bow_vec = bow_vec_orig[:]
label = label_orig[:]
# the training loop
for epoch in range(10):
    start = time.time()
    for ind, instance in enumerate(train_data_X):
        task1b.zero_grad()
        probs = task1b(bow_vec[ind]) # forward pass
        loss = loss_function1b(probs, label[ind])
        loss.backward()
        opt1b.step()
    end = time.time()
    print('Time Taken - ' + str(end - start))
    print('Epoch - ' + str(epoch) + ' , LOSS - ' + str(loss.data))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Time Taken - 16.428025484085083
Epoch - 0 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.185876846313477
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.273927688598633
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.31970238685608
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.358357906341553
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.48376703262329
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.91481351852417
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.292543649673462
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.265172481536865
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.27046847343445
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')


*TESTING*

In [65]:
print('--- AFTER TRAINING ---')
countPredictedNegatives = 0
countTotalCorrect = 0
countActualPositives = 0
countPredictedPostives = 0

for ind, instance in enumerate(test_data_X):
    bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
    logprobs = task1b(bow_vec)
    pred = np.argmax(logprobs.cpu().data.numpy())
    label = test_data_Y[ind]
    if label == 1:
      countActualPositives += 1
      if ix_to_label[pred] == 1:
        countPredictedPostives += 1
      else:
        countPredictedNegatives += 1
      
    if label == ix_to_label[pred]:
      countTotalCorrect += 1
      
print('Accuracy' + str(float(countTotalCorrect)/ len(test_data_X)))
print('Recall' + str(float(countPredictedPostives)/ countActualPositives))
#print('Precision - ' + str(float(countPredictedPostives)/ (countPredictedPostives + countPredictedNegatives)))

--- AFTER TRAINING ---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Accuracy0.8318
Recall0.7585385226370135
Precision - 0.7585385226370135


Save the model

In [0]:
import torch
torch.save(bow,'model1b_gpu.json')

from google.colab import files
model = files.download("model1b_gpu.json")

Task 1 c

In [0]:
# the training loop
for epoch in range(10):
    start = time.time()
    for ind, instance in enumerate(train_data_X):
        task1c.zero_grad()
        probs = task1c(bow_vec[ind]) # forward pass
        loss = loss_function1c(probs, label[ind])
        loss.backward()
        opt1c.step()
    end = time.time()
    print('Time Taken - ' + str(end - start))
    print('Epoch - ' + str(epoch) + ' , LOSS - ' + str(loss.data))

In [0]:
print('--- AFTER TRAINING ---')
countPredictedNegatives = 0
countTotalCorrect = 0
countActualPositives = 0
countPredictedPostives = 0

for ind, instance in enumerate(test_data_X):
    bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
    logprobs = task1c(bow_vec)
    pred = np.argmax(logprobs.cpu().data.numpy())
    label = test_data_Y[ind]
    if label == 1:
      countActualPositives += 1
      if ix_to_label[pred] == 1:
        countPredictedPostives += 1
      else:
        countPredictedNegatives += 1
      
    if label == ix_to_label[pred]:
      countTotalCorrect += 1
      
print('Accuracy' + str(float(countTotalCorrect)/ len(test_data_X)))
print('Recall' + str(float(countPredictedPostives)/ countActualPositives))
print('Precision' + str(float(countPredictedPostives)/ countPredictedPostives + countPredictedNegatives))

--- AFTER TRAINING ---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


0.8436


Save the model

In [0]:
import torch
torch.save(bow,'model1c_gpu.json')

from google.colab import files
model = files.download("model1c_gpu.json")

In [57]:
from google.colab import files
temp_test = files.upload()


Saving model.json to model (1).json


In [0]:
import torch
bow = torch.load(io.BytesIO(temp_test['model.json']))

In [59]:
from google.colab import files
temp_test = files.upload()

Saving word_to_ix.pickle to word_to_ix (1).pickle


In [0]:
pickle_in = open("word_to_ix.pickle","rb")
word_to_ix = pickle.load(pickle_in)